<a href="https://colab.research.google.com/github/robin9804/Openholo_Val/blob/master/Depth_map_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Depthmap 기반 홀로그램 생성

fresnel integral 기반 back propagation

In [ ]:
# parameters
mm = 10e-3
um = mm*mm
nm = um*mm
wvl_R = 639 * nm  # Red
wvl_G = 525 * nm  # Green
wvl_B = 463 * nm  # Blue

def delta(wvl):
    return wvl * 10  # sampling period

def k0(wvl):
    return (np.pi * 2) / wvl  # wave number

pp = 3.45 * um  # pixel to pixel parameter
z_scale = 0.3  # depth range to real scale

# resolution setting
width = 1920 
height = 1080

# Depthmap file read

depth map bmp file을 읽어오는 과정

In [ ]:
depthfile = 'file path'
rgbfile = 'file path'
with open(depthfile) as f:
    depthmap = np.array(f)

with open(rgbfile) as f:
    rgbmap = np.array(f)

alphamap = np.zeros((width, height))
for i in range(width):
    for j in range(height):
        if depthmap[i,j] > 0:
            alphamap[i,j] = 1

# Back propagation 을 사용하여 hologram 취득


In [ ]:
def h_Fresnel(x1, y1, z1, x2, y2, z2, wvl):
    """
    Fresnel integral inpulse response
    """
    z = abs(z2 - z1)
    x = x1 - x2
    y = y1 - y2
    a = np.exp(1j* k0(wvl) * z)
    h = a * np.exp(1j*(k0(wvl)/(2*z))*(x**2 + y**2))
    return h

def gm(n, wvl):
    """
    n : order
    x, y: coordinate
    """
    g1 = np.zeros((height, width))
    for i in range(width):
        for j in range(height):
            if depthmap[i,j] == n:
                x = i - width/2
                y = j - height/2
                g1[i,j] = rgbmap[i,j] * np.exp(1j*k0(wvl) * alphamap[i,j] - g(m-1))

In [ ]:
# propagation with no occlusion
def Tm(n, z, wvl):
    if n = 1:
        for i in zip(depthmap):
            if i == 1:
                return np.exp(1j*k0(wvl)*alphamap())

In [ ]:
M = 255 # depth level
m = 1
while m == (M-1):
    m += 1
    